In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from airo_camera_toolkit.utils import ImageConverter
from airo_dataset_tools.data_parsers.pose import Pose
from airo_camera_toolkit.calibration.fiducial_markers import draw_frame_on_image
from airo_spatial_algebra.se3 import SE3Container


np.set_printoptions(precision=3, suppress=True)

In [ ]:
pose_id = 3
image = cv2.imread(f"output/pose_{pose_id:04d}.png")

image_rgb = ImageConverter.from_opencv_format(image).image_in_numpy_format
plt.imshow(image_rgb);

In [ ]:
eef_pose = Pose.parse_file(f"output/eef_pose_{pose_id:04d}.json").as_homogeneous_matrix()
camera_pose = Pose.parse_file("output/camera_extrinsics.json").as_homogeneous_matrix()
eef_to_board = Pose.parse_file("output/eef_to_board_transform.json").as_homogeneous_matrix()

print(eef_pose)
print(camera_pose)

In [ ]:
from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics

# Load camera intrinsics from file
intrinsics_saved = CameraIntrinsics.parse_file("camera_intrinsics.json")

def intrinsics_to_K_matrix(intrinsics: CameraIntrinsics):
    fx = intrinsics.focal_lengths_in_pixels.fx
    fy = intrinsics.focal_lengths_in_pixels.fy
    cx = intrinsics.principal_point_in_pixels.cx
    cy = intrinsics.principal_point_in_pixels.cy
    return np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

intrinsics_matrix = intrinsics_to_K_matrix(intrinsics_saved)

# with np.printoptions(precision=3, suppress=True):
print(intrinsics_matrix)

In [ ]:
# W = world
# C = camera
# E = end effector
# B = board

X_WC = camera_pose
X_WE = eef_pose
X_CW = np.linalg.inv(X_WC)
X_CE = X_CW @ X_WE
X_EB = eef_to_board
X_CB = X_CE @ X_EB

image2 = image.copy()
#draw_frame_on_image(image2, X_CE, intrinsics_matrix)
charuco_se3 = SE3Container.from_homogeneous_matrix(X_CB)
rvec = charuco_se3.orientation_as_rotation_vector
tvec = charuco_se3.translation
cv2.drawFrameAxes(image2, intrinsics_matrix, None, rvec, tvec, 0.2)


image2_rgb = ImageConverter.from_opencv_format(image2).image_in_numpy_format
plt.imshow(image2_rgb);

In [ ]:
from airo_camera_toolkit.calibration.fiducial_markers import (
    AIRO_DEFAULT_ARUCO_DICT,
    AIRO_DEFAULT_CHARUCO_BOARD,
    detect_aruco_markers,
    detect_charuco_corners,
    draw_frame_on_image,
    get_pose_of_charuco_board,
    visualize_charuco_detection,
)

aruco_dict = AIRO_DEFAULT_ARUCO_DICT
charuco_board = AIRO_DEFAULT_CHARUCO_BOARD

def detect_and_draw_charuco_pose(image, intrinsics_matrix):
    aruco_result = detect_aruco_markers(image, aruco_dict)
    if not aruco_result:
        return

    charuco_result = detect_charuco_corners(image, aruco_result, charuco_board)
    if not charuco_result:
        return

    image = visualize_charuco_detection(image, charuco_result)
    charuco_pose = get_pose_of_charuco_board(charuco_result, charuco_board, intrinsics_matrix, None)
    if charuco_pose is None:
        return

    charuco_se3 = SE3Container.from_homogeneous_matrix(charuco_pose)
    rvec = charuco_se3.orientation_as_rotation_vector
    tvec = charuco_se3.translation
    cv2.drawFrameAxes(image, intrinsics_matrix, None, rvec, tvec, 0.2)
    return charuco_pose


image3 = image.copy()
charuco_pose = detect_and_draw_charuco_pose(image3, intrinsics_matrix)
image3_rgb = ImageConverter.from_opencv_format(image3).image_in_numpy_format
plt.imshow(image3_rgb);

In [ ]:
from airo_typing import HomogeneousMatrixType

def translational_error(pose: HomogeneousMatrixType, ground_truth: HomogeneousMatrixType):
    return np.linalg.norm(pose[:3, 3] - ground_truth[:3, 3])

translational_error(X_CB, charuco_pose)

In [ ]:
R1 = X_CB[:3, :3]
R2 = charuco_pose[:3, :3]

import scipy

def rotational_error(R1, R2):
    # phi_6 from: Metrics for 3D Rotations: Comparison and Analysis
    return np.linalg.norm(scipy.linalg.logm(R1 @ R2.T))

r = rotational_error(R1, R2)
r, np.rad2deg(r)